In [1]:
import numpy as np
import qilisdk
import qutip as qt
import matplotlib.pyplot as plt
import sys

sys.path.append('/Users/omichel/Desktop/qilimanjaro/projects/retech/retech_2025/src')

import hamiltonian
import dynamics
import plots
import estimator
import VQE

from qilisdk.backends import QutipBackend
from qilisdk.digital import Circuit, M, U1, CNOT, U2, U3, CZ, RX, RZ, H
from qilisdk.digital.ansatz import HardwareEfficientAnsatz
from qilisdk.optimizers import SciPyOptimizer
from qilisdk.functionals.variational_program import VariationalProgram
from qilisdk.functionals.sampling import Sampling, SamplingResult
from qilisdk.functionals.time_evolution import TimeEvolution
from scipy.optimize import minimize

from qilisdk.core import Model, BinaryVariable
from qilisdk.cost_functions import ModelCostFunction

In [ ]:
nqubits = 2

H_test = hamiltonian.create_random_hamiltonian(nqubits, -1.0, 1.0)
psi_0 = dynamics.initialize_down_state(nqubits)

print(H_test.H)

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=CSR, isherm=True
Qobj data =
[[ 0.70628703  0.36933522 -0.27708757  0.        ]
 [ 0.36933522  0.0721904   0.         -0.27708757]
 [-0.27708757  0.          1.22052014  0.36933522]
 [ 0.         -0.27708757  0.36933522 -1.99899757]]


In [3]:
ti = 0
tf = 2
nsteps = 1000

sim = dynamics.time_evolution(H_test, psi_0, ti, tf, nsteps)

observables = dynamics.calculate_expectation_values(sim, H_test)

print(observables['Sz'][-1])
print(observables['Sx'][-1])

target_state = sim.states[-1]

analog_probabilities = dynamics.get_bitstring_probabilities(target_state, basis='Z')

print(analog_probabilities)


[0.53735031 0.29564319]
[0.2184186  0.55492031]
{'00': 0.4319954028128188, '01': 0.3366797499636281, '10': 0.21582619419638213, '11': 0.015498653027171055}


In [ ]:
layers = 1
backend= QutipBackend

connectivity = VQE.generate_connectivity_list(nqubits, mode = 'ATA')
per_layer = 2*nqubits + len(connectivity)
params_uniform = np.random.uniform(-np.pi, np.pi, size=(layers * per_layer))
params_zero = np.zeros(layers * per_layer)

# minimize with scipy 
res = minimize(lambda p: VQE.nll_cost(p, nqubits, layers, connectivity, analog_probabilities, backend=backend),
               params_zero, method='COBYLA',
               options={'maxiter': 200})
print("Optimization done, success:", res.success, "fun:", res.fun)
print("best params:", res.x)